In [1]:
import torch


def shock_index(data):
    """ HR/SBP ratio. """
    return data['HR'] / data['SBP']


def age_normalised_shock_index(data):
    """ HR/(SBP * Age). """
    return data['HR'] / (data['SBP'] * data['Age'])


def modfied_shock_index(data):
    """ HR/(SBP * Age). """
    return data['HR'] / (data['MAP'] * data['Age'])


def bun_cr(data):
    """ BUN/Creatinine. """
    return data['BUN'] / data['Creatinine']


def sao2_fio2(data):
    """ SaO2/FiO2. """
    return data['SaO2'] / data['FiO2']


def urea_creatinine(data):
    """ BUN/Creatinine. """
    return data['BUN'] / data['Creatinine']


def pulse_pressure(data):
    """ SBP - DBP. """
    return data['SBP'] - data['DBP']


def cardiac_output(data):
    """ Pulse pressure * heart_rate. """
    pp = data['SBP'] - data['DBP']
    return pp * data['HR']


def partial_sofa(data):
    """ Partial reconstruction of the SOFA score from features available in the sepsis dataset. """
    # Init the tensor
    N, L, C = data.data.size()
    sofa = torch.zeros(N, L, 1)

    # Coagulation
    platelets = data['Platelets']
    sofa[platelets >= 150] += 0
    sofa[(100 <= platelets) & (platelets < 150)] += 1
    sofa[(50 <= platelets) & (platelets < 100)] += 2
    sofa[(20 <= platelets) & (platelets < 50)] += 3
    sofa[platelets < 20] += 4

    # Liver
    bilirubin = data['Bilirubin_total']
    sofa[bilirubin < 1.2] += 0
    sofa[(1.2 <= bilirubin) & (bilirubin <= 1.9)] += 1
    sofa[(1.9 < bilirubin) & (bilirubin <= 5.9)] += 2
    sofa[(5.9 < bilirubin) & (bilirubin <= 11.9)] += 3
    sofa[bilirubin > 11.9] += 4

    # Cardiovascular
    map = data['MAP']
    sofa[map >= 70] += 0
    sofa[map < 70] += 1

    # Creatinine
    creatinine = data['Creatinine']
    sofa[creatinine < 1.2] += 0
    sofa[(1.2 <= creatinine) & (creatinine <= 1.9)] += 1
    sofa[(1.9 < creatinine) & (creatinine <= 3.4)] += 2
    sofa[(3.4 < creatinine) & (creatinine <= 4.9)] += 3
    sofa[creatinine > 4.9] += 4

    return 

In [2]:
def features_score(data):
    dat = data[-1]
    news_score = np.zeros((1, 8))

    HR = dat[0]
    if HR == np.nan:
        HR_score = np.nan
    elif (HR <= 40) | (HR >= 131):
        HR_score = 3
    elif 111 <= HR <= 130:
        HR_score = 2
    elif (41 <= HR <= 50) | (91 <= HR <= 110):
        HR_score = 1
    else:
        HR_score = 0
    news_score[0, 0] = HR_score

    Temp = dat[2]
    if Temp == np.nan:
        Temp_score = np.nan
    elif Temp <= 35:
        Temp_score = 3
    elif Temp >= 39.1:
        Temp_score = 2
    elif (35.1 <= Temp <= 36.0) | (38.1 <= Temp <= 39.0):
        Temp_score = 1
    else:
        Temp_score = 0
    news_score[0, 1] = Temp_score

    Resp = dat[6]
    if Resp == np.nan:
        Resp_score = np.nan
    elif (Resp < 8) | (Resp > 25):
        Resp_score = 3
    elif 21 <= Resp <= 24:
        Resp_score = 2
    elif 9 <= Resp <= 11:
        Resp_score = 1
    else:
        Resp_score = 0
    news_score[0, 2] = Resp_score

    Creatinine = dat[19]
    if Creatinine == np.nan:
        Creatinine_score = np.nan
    elif Creatinine < 1.2:
        Creatinine_score = 0
    elif Creatinine < 2:
        Creatinine_score = 1
    elif Creatinine < 3.5:
        Creatinine_score = 2
    else:
        Creatinine_score = 3
    news_score[0, 3] = Creatinine_score

    MAP = dat[4]
    if MAP == np.nan:
        MAP_score = np.nan
    elif MAP >= 70:
        MAP_score = 0
    else:
        MAP_score = 1
    news_score[0, 4] = MAP_score

    SBP = dat[3]
    if SBP + Resp == np.nan:
        qsofa = np.nan
    elif (SBP <= 100) & (Resp >= 22):
        qsofa = 1
    else:
        qsofa = 0
    news_score[0, 5] = qsofa

    Platelets = dat[33]
    if Platelets == np.nan:
        Platelets_score = np.nan
    elif Platelets <= 50:
        Platelets_score = 3
    elif Platelets <= 100:
        Platelets_score = 2
    elif Platelets <= 150:
        Platelets_score = 1
    else:
        Platelets_score = 0
    news_score[0, 6] = Platelets_score

    Bilirubin = dat[26]
    if Bilirubin == np.nan:
        Bilirubin_score = np.nan
    elif Bilirubin < 1.2:
        Bilirubin_score = 0
    elif Bilirubin < 2:
        Bilirubin_score = 1
    elif Bilirubin < 6:
        Bilirubin_score = 2
    else:
        Bilirubin_score = 3
    news_score[0, 7] = Bilirubin_score

    news_score = news_score[-1]
    return news_score